# Version 2

---


In [ ]:
# Part 1
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy.stats import gaussian_kde
import gc

# visualization defaults
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')
sns.set_palette("viridis")

# Check for GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("GPU Available:", len(physical_devices) > 0)
if len(physical_devices) > 0:
    print("GPU Devices:", physical_devices)
    
    # Set memory growth for better GPU memory utilization
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    try:
        tf.config.optimizer.set_jit(True)
        print("XLA JIT compilation enabled")
    except:
        print("XLA JIT compilation not available")

# Set to float32 for stability
tf.keras.mixed_precision.set_global_policy('float32')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0005,  # Reduced learning rate for stability
    clipnorm=0.5,          # Use gradient clipping
    epsilon=1e-7
)

output_dir = os.path.join('private', 'data', 'training_data')
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'visualizations'), exist_ok=True)

# Load data
print("Loading data...")
input_file_path = os.path.join('private', 'data', 'engineered', 'weekly_sequence_engineered.parquet')
df = pd.read_parquet(input_file_path)
gc.collect()  # Free memory after loading data

print("Available columns in the DataFrame:")
print(df.columns.tolist())

# Issues with year
# Process year info
if 'Year' not in df.columns:
    if 'WeekStart' in df.columns:
        try:
            if not pd.api.types.is_datetime64_any_dtype(df['WeekStart']):
                df['WeekStart'] = pd.to_datetime(df['WeekStart'])
            df['Year'] = df['WeekStart'].dt.year
            print("Created 'Year' column from 'WeekStart'")
        except Exception as e:
            print(f"Could not create 'Year' column from 'WeekStart': {e}")
    else:
        print("'Year' column not found and no 'WeekStart' column available. Skipping year filtering.")

# Filter by Year if it exists
if 'Year' in df.columns:
    recent_years = [2022, 2023, 2024]
    df = df[df['Year'].isin(recent_years)].copy()
    print(f"Filtered to {len(df)} rows from years {recent_years}")
else:
    print("Cannot filter by year, continuing with full dataset")


# Add enhanced features - optimized and stabilized
def add_enhanced_features(df):    
    # Check if WeekStart is a datetime type
    if 'WeekStart' in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df['WeekStart']):
            df['WeekStart'] = pd.to_datetime(df['WeekStart'])
    
    # Create time features
    if 'Month' not in df.columns and 'WeekStart' in df.columns:
        try:
            # Extract month and quarter from WeekStart
            df['Month'] = df['WeekStart'].dt.month
            df['Quarter'] = df['WeekStart'].dt.quarter
        except Exception as e:
            print(f"Could not create time columns from 'WeekStart': {e}")
    
    # Cyclical encoding
    if 'Month' in df.columns:
        df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
        df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)
    else:
        print("'Month' column not available, skipping month cyclical encoding")
    
    if 'Quarter' in df.columns:
        df['quarter_sin'] = np.sin(2 * np.pi * df['Quarter']/4)
        df['quarter_cos'] = np.cos(2 * np.pi * df['Quarter']/4)
    else:
        print("'Quarter' column not available, skipping quarter cyclical encoding")
    
    # Calculate maintenance rate
    if all(col in df.columns for col in ['ReceivedCount', 'RunningFleetTotal']):
        # Clip to prevent outliers
        received_clipped = np.clip(df['ReceivedCount'], 0, df['ReceivedCount'].quantile(0.99))
        fleet_clipped = np.maximum(df['RunningFleetTotal'], 1)
        df['maintenance_rate'] = received_clipped / fleet_clipped
    else:
        print("Required columns for maintenance_rate calculation not available")
    
    # Add volatility measure
    if 'ReceivedCount' in df.columns:
        received_clipped = df.groupby('PartNumber')['ReceivedCount'].transform(
            lambda x: np.clip(x, 0, x.quantile(0.99) if len(x) > 0 else np.max(x))
        )
        
        # Calculate rolling stats
        for window in [4, 12]:
            col_name = f'ReceivedCount_roll{window}w_std'
            if col_name not in df.columns:
                try:
                    df[col_name] = df.groupby('PartNumber')['ReceivedCount'].transform(
                        lambda x: x.rolling(window, min_periods=1).std()
                    ).fillna(0)
                    
                    # Cap to prevent extreme values
                    std_99th = df[col_name].quantile(0.99)
                    df[col_name] = df[col_name].clip(0, std_99th)
                except Exception as e:
                    print(f"Could not calculate rolling std for window {window}: {e}")
    
    return df

# Add enhanced features
df = add_enhanced_features(df)
gc.collect() 

print("Filtering parts with minimum maintenance events...")
min_repairs = 10

# More efficient part filtering
part_maintenance_sums = df.groupby('PartNumber')['ReceivedCount'].sum()
parts_with_maintenance = part_maintenance_sums[part_maintenance_sums >= min_repairs].index

print(f"Parts with at least {min_repairs} maintenance events: {len(parts_with_maintenance)} out of {len(part_maintenance_sums)} total parts")
df = df[df['PartNumber'].isin(parts_with_maintenance)].copy()
print(f"Dataset now has {len(df)} rows after filtering to parts with substantial maintenance history")
gc.collect() 

# Enhanced feature selection with focus on stability
enhanced_features = [
    # Core identifiers
    'PartNumber', 'Year', 'Month', 'Quarter', 'WeekStart',
    'ReceivedCount',  # Target
    # Original important features
    'RunningFleetTotal',
    'ReceivedCount_roll12w_mean',
    'ReceivedCount_lag1w',
    'ShippedCount',
    'InService',
    # New cyclical time features
    'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
    # Volatility features
    'ReceivedCount_roll4w_std', 'ReceivedCount_roll12w_std',
    'maintenance_rate'
]

existing_features = [col for col in enhanced_features if col in df.columns]
print(f"Using {len(existing_features)} features: {existing_features}")
reduced_df = df[existing_features].copy()
del df  # Delete original dataframe to free memory
gc.collect()  

print("Checking and cleaning data...")
numeric_cols = reduced_df.select_dtypes(include=['number']).columns

print("Applying log(x+1) transform to target for better stability")
reduced_df['ReceivedCount_original'] = reduced_df['ReceivedCount'].copy()  # Save original for later
reduced_df['ReceivedCount'] = np.log1p(reduced_df['ReceivedCount'])  # log(x+1) transformation

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
sns.histplot(reduced_df['ReceivedCount_original'], bins=30, kde=True, color='blue')
plt.title('Original Maintenance Event Distribution')
plt.xlabel('Maintenance Events Count')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
sns.histplot(reduced_df['ReceivedCount'], bins=30, kde=True, color='green')
plt.title('Log-Transformed Maintenance Event Distribution')
plt.xlabel('Log(Maintenance Events + 1)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'visualizations', 'target_transformation.png'))
plt.close()

# Find and replace outliers
for col in numeric_cols:
    if col in ['Year', 'Month', 'Quarter', 'PartNumber', 'time_id']:
        continue  # Skip identifier columns
    
    # Detect infinities
    inf_mask = np.isinf(reduced_df[col])
    if inf_mask.any():
        print(f"{inf_mask.sum()} infinite values detected in {col} - replacing")
        reduced_df.loc[inf_mask, col] = np.nan
    
    # Detect NaNs
    nan_mask = np.isnan(reduced_df[col])
    if nan_mask.any():
        print(f"{nan_mask.sum()} NaN values detected in {col} - filling with 0")
        reduced_df.loc[nan_mask, col] = 0
    
    # Handle extreme values
    if col != 'ReceivedCount' and col != 'ReceivedCount_original':  # Don't clip target
        Q1 = reduced_df[col].quantile(0.25)
        Q3 = reduced_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        
        outliers = (reduced_df[col] < lower_bound) | (reduced_df[col] > upper_bound)
        if outliers.any():
            print(f"{outliers.sum()} outliers detected in {col} - capping to IQR bounds")
            reduced_df.loc[outliers, col] = np.clip(
                reduced_df.loc[outliers, col], lower_bound, upper_bound)

# Create temporal identifier for sorting
reduced_df['time_id'] = reduced_df['Year'] * 100 + reduced_df['Month']

# Analyze time distribution
print("\nAnalyzing time distribution:")
time_counts = reduced_df['time_id'].value_counts().sort_index()
print(f"Earliest time period: {time_counts.index.min()}")
print(f"Latest time period: {time_counts.index.max()}")
print(f"Number of time periods: {len(time_counts)}")

# Prepare Time Based Sequences
def prepare_time_based_sequences(df, sequence_length=8, target_column='ReceivedCount', batch_size=32,
                                train_ratio=0.7, val_ratio=0.15):
    
    # Get numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col != target_column 
                   and col != 'time_id' 
                   and col != 'ReceivedCount_original']
    
    # Sort by time
    df = df.sort_values(['PartNumber', 'time_id'])
    # Get unique time periods
    unique_times = np.sort(df['time_id'].unique())
    # Calculate split points
    if len(unique_times) < 10:
        raise ValueError("Not enough time periods for splitting")
        
    train_split = int(len(unique_times) * train_ratio)
    val_split = int(len(unique_times) * (train_ratio + val_ratio))
    
    train_times = unique_times[:train_split]
    val_times = unique_times[train_split:val_split]
    test_times = unique_times[val_split:]
    
    print(f"\nTime-based split:")
    print(f"Training periods: {len(train_times)} periods from {train_times[0]} to {train_times[-1]}")
    print(f"Validation periods: {len(val_times)} periods from {val_times[0]} to {val_times[-1]}")
    print(f"Test periods: {len(test_times)} periods from {test_times[0]} to {test_times[-1]}")
    
    # Create train/val/test datasets
    train_mask = np.isin(df['time_id'], train_times)
    val_mask = np.isin(df['time_id'], val_times)
    test_mask = np.isin(df['time_id'], test_times)
    
    train_df = df[train_mask].copy()
    val_df = df[val_mask].copy()
    test_df = df[test_mask].copy()
    
    print(f"\nRows in each split:")
    print(f"Training set: {len(train_df)} rows ({len(train_df) / len(df):.1%})")
    print(f"Validation set: {len(val_df)} rows ({len(val_df) / len(df):.1%})")
    print(f"Test set: {len(test_df)} rows ({len(test_df) / len(df):.1%})")
    
    # Check part coverage
    train_parts = set(train_df['PartNumber'].unique())
    val_parts = set(val_df['PartNumber'].unique())
    test_parts = set(test_df['PartNumber'].unique())
    
    print(f"\nPart coverage:")
    print(f"Parts in training set: {len(train_parts)}")
    print(f"Parts in validation set: {len(val_parts)}")
    print(f"Parts in test set: {len(test_parts)}")
    
    # Find parts that don't appear in all splits
    missing_in_val = train_parts - val_parts
    missing_in_test = train_parts - test_parts
    
    print(f"Parts in training but not in validation: {len(missing_in_val)} ({len(missing_in_val) / len(train_parts):.1%})")
    print(f"Parts in training but not in test: {len(missing_in_test)} ({len(missing_in_test) / len(train_parts):.1%})")

    # sequence creation with better error checking
    def create_sequences_from_df(input_df, max_sequences_per_part=None):
        X_sequences = []
        y_values = []
        y_original_values = []  # Store original (non-transformed) values
        part_ids = []
        time_periods = []  # Add this line to store time periods
        
        # Process in batches
        unique_parts = input_df['PartNumber'].unique()
        batch_size = 200
        
        # Create batches
        for batch_idx in range(0, len(unique_parts), batch_size):
            batch_parts = unique_parts[batch_idx:batch_idx + batch_size]
            batch_df = input_df[input_df['PartNumber'].isin(batch_parts)]
            
            for part_id in batch_parts:
                part_data = batch_df[batch_df['PartNumber'] == part_id].sort_values('time_id')
                
                if len(part_data) <= sequence_length:
                    continue
                
                # Extract features and target
                features = part_data[numeric_cols].values
                targets = part_data[target_column].values
                original_targets = part_data['ReceivedCount_original'].values
                time_ids = part_data['time_id'].values  # Get time ids for each row
                
                # Additional validation for infinite or NaN values
                if np.isnan(features).any() or np.isinf(features).any():
                    print(f"NaN/Inf values found in features for part {part_id}. Skipping.")
                    continue
                    
                if np.isnan(targets).any() or np.isinf(targets).any():
                    print(f"NaN/Inf values found in targets for part {part_id}. Skipping.")
                    continue
                
                # Check for unreasonable feature values
                if np.abs(features).max() > 1e4:
                    print(f"Extreme feature values found for part {part_id}. Capping to ±1e4.")
                    features = np.clip(features, -1e4, 1e4)
                
                # Create sequences with a limit per part
                count = 0
                for i in range(len(part_data) - sequence_length):
                    X_sequences.append(features[i:i+sequence_length])
                    y_values.append(targets[i+sequence_length])
                    y_original_values.append(original_targets[i+sequence_length])
                    part_ids.append(part_id)
                    time_periods.append(time_ids[i+sequence_length])  # Store the prediction time period
                    
                    count += 1
                    if max_sequences_per_part and count >= max_sequences_per_part:
                        break
            
            del batch_df
            
            # Clean memory every 500 batches
            if batch_idx % 500 == 0 and batch_idx > 0:
                gc.collect()
                print(f"Processed {batch_idx}/{len(unique_parts)} parts")
        
        if not X_sequences:
            return None, None, None, None, None
        
        X = np.array(X_sequences, dtype=np.float32)
        y = np.array(y_values, dtype=np.float32)
        y_original = np.array(y_original_values, dtype=np.float32)
        part_ids_array = np.array(part_ids)
        time_periods_array = np.array(time_periods)
        
        # Replace any remaining NaN/Inf with zeros
        X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
        
        # Clear lists
        del X_sequences, y_values, y_original_values
        gc.collect()
        
        return X, y, y_original, part_ids_array, time_periods_array
    
    # Create sequences for each split even distribution of parts
    max_sequences_per_part = 100 
    
    print("Creating training sequences...")
    X_train, y_train, y_train_original, train_part_ids, train_time_periods = create_sequences_from_df(
        train_df, max_sequences_per_part)
    del train_df  # Free memory
    gc.collect()
    
    print("Creating validation sequences...")
    X_val, y_val, y_val_original, val_part_ids, val_time_periods = create_sequences_from_df(
        val_df, max_sequences_per_part)
    del val_df  # Free memory
    gc.collect()
    
    print("Creating test sequences...")
    X_test, y_test, y_test_original, test_part_ids, test_time_periods = create_sequences_from_df(
        test_df, max_sequences_per_part)
    del test_df  # Free memory
    gc.collect()
    
    # Check if we have sequences
    if X_train is None or X_val is None or X_test is None:
        raise ValueError("One or more data splits resulted in no sequences")
    
    print(f"\nSequence counts:")
    print(f"Training set: {len(X_train)} sequences")
    print(f"Validation set: {len(X_val)} sequences")
    print(f"Test set: {len(X_test)} sequences")
    
    print(f"\nTransformed target statistics:")
    print(f"Train targets: min={y_train.min():.2f}, max={y_train.max():.2f}, mean={y_train.mean():.2f}, median={np.median(y_train):.2f}")
    print(f"Validation targets: min={y_val.min():.2f}, max={y_val.max():.2f}, mean={y_val.mean():.2f}, median={np.median(y_val):.2f}")
    print(f"Test targets: min={y_test.min():.2f}, max={y_test.max():.2f}, mean={y_test.mean():.2f}, median={np.median(y_test):.2f}")
    
    print(f"\nOriginal target statistics:")
    print(f"Train targets: min={y_train_original.min():.2f}, max={y_train_original.max():.2f}, mean={y_train_original.mean():.2f}, median={np.median(y_train_original):.2f}")
    
    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.cache()
    train_dataset = train_dataset.shuffle(buffer_size=min(10000, len(X_train)))
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
    
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
    
    return (train_dataset, val_dataset, test_dataset, X_train.shape[1:], 
            (X_train, y_train, y_train_original, train_part_ids, train_time_periods), 
            (X_val, y_val, y_val_original, val_part_ids, val_time_periods), 
            (X_test, y_test, y_test_original, test_part_ids, test_time_periods))

# Normalization!
# Normalization using RobustScaler
exclude_from_normalization = ['Year', 'Month', 'Quarter', 'PartNumber', 'ReceivedCount', 'ReceivedCount_original', 'time_id']
numeric_cols = reduced_df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in exclude_from_normalization]

print(f"Normalizing {len(numeric_cols)} numeric columns with RobustScaler")
# Use RobustScaler instead of StandardScaler - better outlier handling
scaler = RobustScaler()
reduced_df[numeric_cols] = scaler.fit_transform(reduced_df[numeric_cols])

# Create sequences
sequence_length = 8
batch_size = 32

print("Creating sequences from the dataset...")
try:
    train_ds, val_ds, test_ds, input_shape, train_data, val_data, test_data = prepare_time_based_sequences(
        reduced_df, sequence_length, 'ReceivedCount', batch_size, train_ratio=0.7, val_ratio=0.15)
    del reduced_df  
    gc.collect()
except Exception as e:
    print(f"Error creating sequences: {e}")
    raise



In [ ]:
# Model Definition
def build_stable_lstm_model(input_shape):
    model = Sequential([
        LSTM(32,  
             activation='tanh',
             recurrent_activation='sigmoid',
             recurrent_regularizer=l2(0.001),
             kernel_regularizer=l2(0.001),
             return_sequences=False,
             input_shape=input_shape),  
        
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(16, 
              activation='relu',
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(1, activation='linear')
    ])
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss='mse',  
        metrics=['mae'] 
    )
    
    return model

# Create and train model
print(f"Building model with input shape: {input_shape}")
model = build_stable_lstm_model(input_shape)
model.summary()

# NaN detection
class NaNCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is not None and (np.isnan(logs.get('loss', 0)) or np.isinf(logs.get('loss', 0))):
            print(f"\nNaN/Inf loss detected at batch {batch}. Terminating training.")
            self.model.stop_training = True
            
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None and (np.isnan(logs.get('loss', 0)) or np.isinf(logs.get('loss', 0))):
            print(f"\nNaN/Inf loss detected at end of epoch {epoch}. Terminating training.")
            self.model.stop_training = True

# Memory cleanup
class MemoryCleanupCallback(tf.keras.callbacks.Callback):
    def __init__(self, cleanup_frequency=3):
        super().__init__()
        self.cleanup_frequency = cleanup_frequency
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.cleanup_frequency == 0:
            gc.collect()
            print(f"Memory cleanup after epoch {epoch}")

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,  # Increased patience for better convergence
    restore_best_weights=True,
    verbose=1
)

# Use ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,  # Increased patience for LR reduction
    min_lr=1e-6,
    verbose=1
)

# Checkpoint to save best model
model_checkpoint = ModelCheckpoint(
    filepath=os.path.join(output_dir, 'time_based_maintenance_model.h5'),
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

nan_callback = NaNCallback()
memory_cleanup = MemoryCleanupCallback(cleanup_frequency=3)

print("Training model...")
try:
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,  # Increased epochs (from 40 to 100)
        callbacks=[
            model_checkpoint, 
            early_stopping, 
            reduce_lr,  # More stable LR reduction
            nan_callback, 
            memory_cleanup
        ],
        verbose=1
    )
    gc.collect()  
    
    # Check if training completed without NaNs
    if history.history['loss'] and any(np.isnan(loss) for loss in history.history['loss']):
        print("NaN values detected in training loss history")
    
    # Save training history
    hist_df = pd.DataFrame(history.history)
    hist_csv_file = os.path.join(output_dir, 'training_history.csv')
    hist_df.to_csv(hist_csv_file, index=False)
    print(f"Training history saved to {hist_csv_file}")
    
    # Save model
    model.save(os.path.join(output_dir, 'time_based_maintenance_model.h5'))
    print(f"Model saved to {os.path.join(output_dir, 'time_based_maintenance_model.h5')}")
    
except Exception as e:
    print(f"Error during training: {e}")
    import traceback
    traceback.print_exc()
    raise

test_data_path = os.path.join(output_dir, 'test_data.npz')
np.savez(test_data_path,
         X_test=test_data[0],
         y_test=test_data[1],
         y_test_original=test_data[2],
         part_ids=test_data[3],  # Add the part IDs
         time_periods=test_data[4])  # Add time period information
print(f"Saved test data to {test_data_path}")
print(f"Saved test data to {test_data_path}")